# 2. Regresión Lineal Regularizada

## ¿Por qué regularizar?

- La mejor forma de entender la regresión regularizada es viendo cómo y por qué se aplica a los mínimos cuadrados ordinarios (OLS).
- En mínimos cuadrados, el objetivo es encontrar el hiperplano (por ejemplo, una línea recta en 2d) que **minimice la suma de los errores al cuadrado** $SSE = \sum (y_i - \hat{y_i})^2$
- Esto significa identificar el hiperplano que minimiza las líneas grises en el gráfico, que representan la distancia vertical entre los valores observados y los valores predichos.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Generamos datos
np.random.seed(42)
Gr_Liv_Area = np.random.uniform(900, 3000, 500)
Sale_Price = 50000 + 100 * Gr_Liv_Area + np.random.normal(0, 50000, 500)

# Ajusto regresipon lineal
coeffs = np.polyfit(Gr_Liv_Area, Sale_Price, 1)
regression_line = np.poly1d(coeffs)

plt.figure(figsize=(10, 5))
plt.scatter(Gr_Liv_Area, Sale_Price, color='red', label="Datos reales")
plt.plot(sorted(Gr_Liv_Area), regression_line(sorted(Gr_Liv_Area)), color='blue', linewidth=2, label="Regresión lineal")
for x, y in zip(Gr_Liv_Area, Sale_Price):
    plt.plot([x, x], [y, regression_line(x)], color='gray', linewidth=0.8)

plt.xlabel("Gr_Liv_Area")
plt.ylabel("Sale_Price")
plt.title("Regresión Lineal de Precios de Venta")
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.show()

- Recordemos entonces que la función objetivo a minimizar es:

$$ \min \left(  \sum_{i=1}^{n}(y_{i}-\hat{y}_{i}(x_i, w))^{2}  \right)$$

- Esta función objetivo funciona bien cuando los datos cumplen con ciertos supuestos clave:  
  1. **Relación lineal** entre las variables.  
  2. **Más observaciones ($n$) que características ($p$)** → es decir, $n>p$.  
  3. **Poca o ninguna multicolinealidad** entre las variables explicativas.

- Muchos datasets reales (ej: data mining o estudios genómicos) tienen muchas más features que muestras.
- A medida que $p$ crece, es más probable que violemos algunos supuestos $\rightarrow$ requiere considerar enfoques alternativos.

### ¿Qué problemas trae un gran número de features?

👎 Menos interpretabilidad: Cuantas más variables, más difícil es entender el modelo.

👎 Infinitas soluciones: Cuando $p>n$, hay infinitas soluciones para el problema de OLS.

- Para abordar esto, aplicamos el principio de _"bet on sparsity"_ (Hastie, Tibshirani y Wainwright, 2015):
    - Asumimos que solo un subconjunto pequeño de características tiene el mayor impacto.

### Selección de Features

####  <font color=purple>1. Hard Thresholding</font>
Selecciona o elimina completamente una feature. Métodos clásicos incluyen:

1. <font color=purple>Selección hacia adelante (Forward Selection)</font>: 
Se empieza con un modelo vacío y se van agregando features una por una, eligiendo la que más mejora el ajuste del modelo.

2. <font color=purple>Eliminación hacia atrás</font>: 
Se empieza con todas las features y se van eliminando una por una, descartando las menos relevantes.

<div style="text-align: center;">
<img src="img/forward-stepwise-algorithm.png" alt="Forward Stepwise Algorithm" width="370">    
<img src="img/backward-stepwise-algorithm.png" alt="Forward Stepwise Algorithm" width="390">
</div>

<font color=red>Desventajas:</font>
- Computacionalmente costoso, sobre todo con muchas variables.
- No escala bien en conjuntos de datos grandes.
- Decisión binaria extrema → una característica está 100% dentro o 100% fuera, sin considerar que puede tener un efecto menor pero relevante.

#### <font color=purple>2. Soft Thresholding</font>

No elimina variables abruptamente, sino que reduce gradualmente sus coeficientes, permitiendo que algunas características pierdan importancia sin ser descartadas de inmediato. 

En algunos casos, esto lleva a que el coeficiente se vuelva exactamente cero, eliminando la variable de manera más natural y optimizada.
    
<font color=purple>Regularización</font>: 
- Objetivo: Restringir el tamaño total de los coeficientes para reducir su magnitud y fluctuación.
- Trade-off: Se reduce la varianza, pero el modelo deja de ser insesgado.

- La función objetivo de un modelo de regresión regularizada agrega un término de penalización P:
$$ \min \left( SSE + P  \right)$$
- Este parámetro de penalización restringe el tamaño de los coeficientes:
    - Los coeficientes solo pueden aumentar si hay una reducción proporcional en el error SSE.

- Este concepto se extiende a otros modelos de la familia de Modelos Lineales Generalizados (como Regresión logística o Regresión de Poisson). 
- Obviamente, cada modelo tiene una función de pérdida diferente, pero el parámetro de penalización funciona igual → restringe los coeficientes a menos que haya una reducción equivalente en la función de pérdida.

Existen tres métodos de penalización comunes: 
1. **Ridge**: Penaliza los coeficientes, pero los mantiene pequeños sin hacerlos cero.
2. **LASSO**: Puede reducir algunos coeficientes a exactamente cero, eliminando variables irrelevantes.
3. **Elastic Net** (ENET): Combinación de Ridge y Lasso, útil cuando hay muchas variables correlacionadas.

### Regresión Ridge
- Controla los coeficientes estimados agregando un término de penalización: $\lambda $ a la función objetivo:

fuente: https://bradleyboehmke.github.io/HOML/regularized-regression.html#ref-hastie2015statistical